In [1]:
import pandas as pd
import numpy as np

## **Combining Datasets: Concat**

Concatenation of $\small\texttt{Series}$ and $\small\texttt{DataFrame}$ objects is very similar to concatenation of NumPy arrays, which can be done via the $\small\texttt{np.concatenate}$ function

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

The first argument is a list or tuple of arrays to concatenate. Additionally, it takes an $\small\texttt{axis}$ keyword that allows you to specify the axis along which the result will be concatenated:

In [ ]:
x = [[1, 2], [3, 4]]
print(np.concatenate([x, x], axis=1))

[[1 2 1 2]
 [3 4 3 4]]


Let's take a look at simple concatenation of $\small\texttt{Series}$ and $\small\texttt{DataFrames}$ with the $\small\texttt{pd.concat}$ function. For convenience, we’ll define this function, which creates a $\small\texttt{DataFrame}$ of a particular form that will be useful below:

In [ ]:
def make_df(cols, ind):
  # Quickly make a DataFrame
  data = {c: [str(c) + str(i) for i in ind]
          for c in cols}
  return pd.DataFrame(data, index=ind)

In [ ]:
print(make_df('ABC', [0, 1, 2]))

    A   B   C
0  A0  B0  C0
1  A1  B1  C1
2  A2  B2  C2


### **Simple Concatenation with pd.concat**

Pandas has a function, $\small\texttt{pd.concat()}$, which has a similar syntax to $\small\texttt{np.concatenate}$ but contains a number of other options. $\small\texttt{pd.concat()}$ can be used for a simple concatenation of $\small\texttt{Series}$ or $\small\texttt{DataFrame}$ objects.

In [ ]:
# Concatenation of Series objects
ser1 = pd.Series(list('ABC'), index=range(1, 4))
ser2 = pd.Series(list('DEF'), index=range(4, 7))
print(pd.concat([ser1, ser2]))

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object


In [ ]:
# Concatenation of DataFrame objects
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


Like $\small\texttt{np.concatenate}$, $\small\texttt{pd.concat}$ allows specification of an axis along which concatenation will take place. By default, the concatenation takes place row-wise within the $\small\texttt{DataFrame}$ (i.e., $\small\texttt{axis=0}$).

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4], axis=1))

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


#### **Duplicate indices**

One important difference between $\small\texttt{np.concatenate}$ and $\small\texttt{pd.concat}$ is that Pandas concatenation *preserves indices*, even if the result will have duplicate indices. While this is valid within $\small\texttt{DataFrames}$, the outcome is often undesirable.

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [3, 4])
y.index = x.index # Make duplicate indices

In [ ]:
print(x); print(y); print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A3  B3
1  A4  B4
    A   B
0  A0  B0
1  A1  B1
0  A3  B3
1  A4  B4


#### **Catching the repeats as an error**

If you’d like to simply verify that the indices in the result of $\small\texttt{pd.concat()}$ do not overlap, you can specify the $\small\texttt{verify_integrity}$ flag. Setting it to $\small\texttt{True}$, the concatenation will raise an exception if there are duplicate indices.

In [ ]:
try:
  pd.concat([x, y], verify_integrity=True)
except ValueError as e:
  print(f'ValueError: {e}')

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### **Ignoring the index**

Sometimes the index itself does not matter, and you would prefer it to simply be ignored. You can do that using the $\small\texttt{ignore_index}$ flag. Setting it to $\small\texttt{True}$, the concatenation will create a new integer index for the resulting $\small\texttt{Series}$:

In [ ]:
print(x); print(y); print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A3  B3
1  A4  B4
    A   B
0  A0  B0
1  A1  B1
2  A3  B3
3  A4  B4


#### **Adding MultiIndex keys**

Another alternative is to use the $\small\texttt{keys}$ option to specify a label for the data sources; the result will be a hierarchically indexed series containing the data:

In [ ]:
print(x); print(y); print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A3  B3
1  A4  B4
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A3  B3
  1  A4  B4


### **Concatenation with joins**

So far, we were mainly concatenating $\small\texttt{DataFrames}$ with shared column names. In practice, data from different sources might have different sets of column names, and $\small\texttt{pd.concat}$ offers several options in this case.

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


By default, the entries for which no data is available are filled with NA values. To change this, we can specify one of several options for the $\small\texttt{join}$ parameter of the concatenate function. By default, the join is a union of the input columns $\small\texttt{(join='outer')}$, but we can change this to an intersection of the columns using $\small\texttt{join='inner'}$.

In [ ]:
print(df5); print(df6); print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


## **Combining Datasets: Merge and Join**

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations. The main interface for this is the $\small\texttt{pd.merge}$ function.

### **Categories of Joins**

The $\small\texttt{pd.merge()}$ function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins. All three types of joins are accessed via an identical call to the $\small\texttt{pd.merge()}$ interface; the type of join performed depends on the form of the input data.

#### **One-to-one joins**

Perhaps the simplest type of merge expression is the one-to-one join, which is in many ways very similar to the column-wise concatenation. To combine the columns of two $\small\texttt{DataFrames}$ into a single $\small\texttt{DataFrame}$, we can use the $\small\texttt{pd.merge()}$ function.

In [ ]:
df1 = pd.DataFrame({'employee': ['John', 'Mary', 'David', 'Emma'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['David', 'John', 'Mary', 'Emma'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)

  employee        group
0     John   Accounting
1     Mary  Engineering
2    David  Engineering
3     Emma           HR
  employee  hire_date
0    David       2004
1     John       2008
2     Mary       2012
3     Emma       2014


In [ ]:
df3 = pd.merge(df1, df2)
print(df3)

  employee        group  hire_date
0     John   Accounting       2008
1     Mary  Engineering       2012
2    David  Engineering       2004
3     Emma           HR       2014


The $\small\texttt{pd.merge()}$ function recognizes that each $\small\texttt{DataFrame}$ has an “employee” column, and automatically joins using this column as a key. The result of the merge is a new $\small\texttt{DataFrame}$ that combines the information from the two inputs.

#### **Many-to-one joins**

Many-to-one joins are joins in which one of the two key columns contains duplicate entries. For the many-to-one case, the resulting $\small\texttt{DataFrame}$ will preserve those duplicate entries as appropriate.

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Gustav', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

  employee        group  hire_date
0     John   Accounting       2008
1     Mary  Engineering       2012
2    David  Engineering       2004
3     Emma           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering     Gustav
2           HR      Steve
  employee        group  hire_date supervisor
0     John   Accounting       2008      Carly
1     Mary  Engineering       2012     Gustav
2    David  Engineering       2004     Gustav
3     Emma           HR       2014      Steve


#### **Many-to-many joins**

If the key column in both the left and right array contains duplicates, then
the result is a many-to-many merge. Consider the following, where we have a $\small\texttt{DataFrame}$ showing one or more skills associated with a particular group.

By performing a many-to-many join, we can recover the skills associated with any individual person:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
print(df1), print(df5), print(pd.merge(df1, df5))

  employee        group
0     John   Accounting
1     Mary  Engineering
2    David  Engineering
3     Emma           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0     John   Accounting          math
1     John   Accounting  spreadsheets
2     Mary  Engineering        coding
3     Mary  Engineering         linux
4    David  Engineering        coding
5    David  Engineering         linux
6     Emma           HR  spreadsheets
7     Emma           HR  organization


(None, None, None)

### **Specification of the Merge Key**

We’ve already seen the default behavior of $\small\texttt{pd.merge()}$: it looks for one or more matching column names between the two inputs, and uses this as the key. However, often the column names will not match so nicely, and $\small\texttt{pd.merge()}$ provides a variety of options to handle this.

#### **The on keyword**

Most simply, you can explicitly specify the name of the key column using the $\small\texttt{on}$ keyword, which takes a column name or a list of column names. This option works only if both the left and right $\small\texttt{DataFrames}$ have the specified column name.

In [ ]:
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

  employee        group
0     John   Accounting
1     Mary  Engineering
2    David  Engineering
3     Emma           HR
  employee  hire_date
0    David       2004
1     John       2008
2     Mary       2012
3     Emma       2014
  employee        group  hire_date
0     John   Accounting       2008
1     Mary  Engineering       2012
2    David  Engineering       2004
3     Emma           HR       2014


#### **The left_on and right_on keywords**

At times you may wish to merge two datasets with different column names; e.g., we may have a dataset in which the employee name is labeled as “name” rather
than “employee”. In this case, we can use the $\small\texttt{left_on}$ and $\small\texttt{right_on keywords}$ to specify the two column names.


In [ ]:
df3 = pd.DataFrame({'name': ['John', 'Mary', 'David', 'Emma'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3);
print(pd.merge(df1, df3, left_on='employee', right_on='name'))

  employee        group
0     John   Accounting
1     Mary  Engineering
2    David  Engineering
3     Emma           HR
    name  salary
0   John   70000
1   Mary   80000
2  David  120000
3   Emma   90000
  employee        group   name  salary
0     John   Accounting   John   70000
1     Mary  Engineering   Mary   80000
2    David  Engineering  David  120000
3     Emma           HR   Emma   90000


The result has a redundant column that we can drop if desired—for example, by
using the $\small\texttt{drop()}$ method of $\small\texttt{DataFrames}$.

In [ ]:
print(pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1))

  employee        group  salary
0     John   Accounting   70000
1     Mary  Engineering   80000
2    David  Engineering  120000
3     Emma           HR   90000


#### **The left_index and right_index keywords**

Sometimes, rather than merging on a column, you would instead like to merge on an index. Let's set the $\small\texttt{employee}$ column as the indices of the $\small\texttt{DataFrames}$, $\small\texttt{df1}$ and $\small\texttt{df2}$ using the $\small\texttt{set_index}$ method.

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a)

                group
employee             
John       Accounting
Mary      Engineering
David     Engineering
Emma               HR
          hire_date
employee           
David          2004
John           2008
Mary           2012
Emma           2014


You can use the index as the key for merging by specifying the $\small\texttt{left_index}$ and/or $\small\texttt{right_index}$ flags in $\small\texttt{pd.merge()}$.

In [ ]:
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group  hire_date
employee                        
John       Accounting       2008
Mary      Engineering       2012
David     Engineering       2004
Emma               HR       2014


For convenience, $\small\texttt{DataFrames}$ implement the $\small\texttt{join()}$ method, which performs a merge that defaults to joining on indices:

In [ ]:
print(df1a); print(df2a); print(df1a.join(df2a))

                group
employee             
John       Accounting
Mary      Engineering
David     Engineering
Emma               HR
          hire_date
employee           
David          2004
John           2008
Mary           2012
Emma           2014
                group  hire_date
employee                        
John       Accounting       2008
Mary      Engineering       2012
David     Engineering       2004
Emma               HR       2014


### **Specifying Set Arithmetic for Joins**

One important consideration in performing a join is the type of set arithmetic used in the join. This comes up when a value appears in one key column but not the other.

In [ ]:
df6 = pd.DataFrame({'name': ['Daniel', 'Peter', 'Jessica'],
                    'food': ['fish', 'beans', 'bread']})
df7 = pd.DataFrame({'name': ['Jessica', 'Joseph'],
                    'drink': ['wine', 'beer']})
print(df6); print(df7); print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


Here we have merged two datasets that have only a single “name” entry in common:
Jessica. By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*. We can specify this explicitly using the $\small\texttt{how}$ keyword, which defaults to $\small\texttt{'inner'}$:

In [ ]:
print(pd.merge(df6, df7, how='inner'))

   name   food drink
0  Mary  bread  wine


Other options for the how keyword are $\small\texttt{'outer'}$, $\small\texttt{'left'}$, and $\small\texttt{'right'}$. An *outer* join returns a join over the union of the input columns, and fills in all missing values with NAs.

In [ ]:
print(df6); print(df7); print(pd.merge(df6, df7, how='outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


The *left join* and *right join* return join over the left entries and right entries, respectively.

In [ ]:
print(df6); print(df7)
print(pd.merge(df6, df7, how='left'))
print(pd.merge(df6, df7, how='right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


### **Overlapping Column Names: The suffixes Keyword**

At times, You may end up in a case where your two input $\small\texttt{DataFrames}$ have conflicting column names.

In [ ]:
df8 = pd.DataFrame({'name': ['David', 'Jacob', 'Jason', 'Danny'],
                   'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['David', 'Jacob', 'Jason', 'Danny'],
                   'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8, df9, on='name'))

    name  rank
0  David     1
1  Jacob     2
2  Jason     3
3  Danny     4
    name  rank
0  David     3
1  Jacob     1
2  Jason     4
3  Danny     2
    name  rank_x  rank_y
0  David       1       3
1  Jacob       2       1
2  Jason       3       4
3  Danny       4       2


Because the output would have two conflicting column names, the merge function
automatically appends a suffix $\small\texttt{_x}$ or $\small\texttt{_y}$ to make the output columns unique. To change the defaults, you can specify a custom suffix using the $\small\texttt{suffixes}$ keyword. These suffixes work in any of the possible join patterns, and work also if there are
multiple overlapping columns.

In [ ]:
print(df8); print(df9)
print(pd.merge(df8, df9, on='name', suffixes=['_L', '_R']))

    name  rank
0  David     1
1  Jacob     2
2  Jason     3
3  Danny     4
    name  rank
0  David     3
1  Jacob     1
2  Jason     4
3  Danny     2
    name  rank_L  rank_R
0  David       1       3
1  Jacob       2       1
2  Jason       3       4
3  Danny       4       2


## **Project: US States Data**

Merge and join operations come up most often when one is combining data from different sources. Here we will consider an example of some data about US states and their populations. The data files $\small\texttt{state-population.csv}$, $\small\texttt{state-areas.csv}$, and $\small\texttt{state-abbrevs.csv}$ can be found in my very GitHub repository.

In [2]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')
print(pop.head()); print(areas.head()); print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


Given this information, say we want to rank US states and territories by their 2010 population density. We will have to combine the datasets to get it. We’ll start with a many-to-one merge that will give us the full state name within the population $\small\texttt{DataFrame}$. We want to merge based on the $\small\texttt{state/region}$ column of $\small\texttt{pop}$, and the $\small\texttt{abbreviation}$ column of $\small\texttt{abbrevs}$. We’ll use $\small\texttt{how='outer'}$ to make sure no data is thrown away due to mismatched labels.

In [4]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # Drop duplicate info
print(merged.head())

  state/region     ages  year  population    state
0           AL  under18  2012   1117489.0  Alabama
1           AL    total  2012   4817528.0  Alabama
2           AL  under18  2010   1130966.0  Alabama
3           AL    total  2010   4785570.0  Alabama
4           AL  under18  2011   1125763.0  Alabama


In [5]:
print(merged.isnull().any()) # Checks whether there were any mismatches

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool


Some of the $\small\texttt{population}$ info is null; let’s figure out which these are!

In [6]:
print(merged[merged['population'].isnull()].head())

     state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN


We see also that some of the new $\small\texttt{state}$ entries are also null, which means that there was no corresponding entry in the $\small\texttt{abbrevs}$ key! Let’s figure out which regions lack this match.

In [7]:
print(merged.loc[merged['state'].isnull(), 'state/region'].unique())

['PR' 'USA']


Our population data includes entries for Puerto Rico
(PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key. We can fix these quickly by filling in appropriate entries.

In [8]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

We see there are no more $\small\texttt{nulls}$ in the state column. Now we can merge the result with the area data using a similar procedure. Examining our results, we will want to join on the $\small\texttt{state}$ column in both.

In [9]:
final = pd.merge(merged, areas, on='state', how='left')
print(final.head())

  state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0


In [10]:
final.isnull().any() # Check for any mismatches

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [11]:
final['state'][final['area (sq. mi)'].isnull()].unique() # See which regions were ignored

array(['United States'], dtype=object)

We see that our $\small\texttt{areas DataFrame}$ does not contain the area of the United States as a whole. We’ll just drop the null values because the population density of the entire United States is not relevant to the current discussion.

In [12]:
final.dropna(inplace=True)
print(final.head())

  state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0


To find the population density of each state, let’s first select the portion of the data corresponding with the year 2010, and the total population. We’ll use the $\small\texttt{query()}$ function to do this quickly.

In [13]:
data2010 = final.query('year == 2010 & ages == \'total\'')
print(data2010.head())

    state/region   ages  year  population       state  area (sq. mi)
3             AL  total  2010   4785570.0     Alabama        52423.0
91            AK  total  2010    713868.0      Alaska       656425.0
101           AZ  total  2010   6408790.0     Arizona       114006.0
189           AR  total  2010   2922280.0    Arkansas        53182.0
197           CA  total  2010  37333601.0  California       163707.0


Now let’s compute the population density and display it in order. We’ll start by reindexing our data on the state, and then compute the result.

In [14]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the least dense states.

In [15]:
print(density.tail())

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64


We see that the least dense state, by far, is Alaska.